# Extra Trees classifier introduction
* this is  a ensemble bagging  method coming from sklearn



In [ ]:
#For Kaggle
#date=''

#For WL data
source='WL'
date='20241118'

saveImage=False

packageName='07-sklearn.ensemble.a-bagging'
classifierName='ExtraTreesClassifier'
extraParameterName='1-Default'

print('done')

In [ ]:
import sys
import os
from importlib import reload
fpath = os.path.join('..//scripts')
sys.path.append(fpath)

import warnings
warnings.filterwarnings('ignore')

#loading internal scripts
import frauddetection as fd
import sourcedata as sd
import dataimport as di
import result as resultMd
reload(fd)
reload(sd)
reload(di)
reload(resultMd)

print('done')

In [ ]:
# Hyperparameters Results

n_estimatorsFound=115
max_depthFound=37
min_samples_splitFound=2
min_samples_leafFound=1

resultMd.update_hyperparameter_config_result(packageName,classifierName,extraParameterName,max_depthFound,n_estimatorsFound)

print('done')

In [ ]:
import pandas as pd

dfTrx = pd.read_csv('../data/cleaned/'+source+'export'+date+'.csv')
dfTrx.head(5)

# Test with default hyperparameters

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = ExtraTreesClassifier(random_state=42)
modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.a-bagging','ExtraTreesClassifier','1-Default', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble-a-bagging','ExtraTreesClassifier','1-Default', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)


In [ ]:
import numpy as np

depth=[]
for tree in modelClf.estimators_:
    depth=np.append(depth, tree.get_depth())
print(np.max(depth))

# Scaling choice
with ensemble algorithm the scaling has no influence

# Hyperparameters tuning

In [ ]:
%%script false

from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import randint

modelClf = ExtraTreesClassifier(random_state=42)
dic_param={
    'n_estimators': randint(90,130),
    'max_depth': randint(15,39),
    'min_samples_leaf': [1],
    'min_samples_split': [2]
}
res=fd.hyperparameterSelectionRandomizedSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None,iter=5)
print(res)

#{'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 108}
#0.24912232751316427
#score   0.9150599270453361
#scoref1 0.9150599270453361

#{'max_depth': 29, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 97}
#0.2519035584310886
#score   0.8960763520678685
#scoref1 0.8960763520678685

#{'max_depth': 35, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 119}
#0.2684369780790882
#score   0.9663032705649157


In [ ]:
%%script false
from sklearn.ensemble import ExtraTreesClassifier

modelClf = ExtraTreesClassifier(random_state=42)

dic_param={
    'n_estimators': [114,115,116,117,118],
    'max_depth': [35,36,37,38,39],
    'min_samples_leaf': [1],
    'min_samples_split': [2]
}
res=fd.hyperparameterSelectionGridSearchCV(modelClf, dic_param, 'f1', dfTrx, predictors, [], None)
print(res)

#{'max_depth': 29, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 120}
#0.2537067302221238
#scoref1 0.8949044585987261

#{'max_depth': 37, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 118}
#0.2735579082959772
#scoref1 0.9754178957718781

#{'max_depth': 37, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 116}
#0.2738991148336759
#scoref1 0.9754178957718781

#{'max_depth': 37, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 115}
#0.27559378583252736
#scoref1 0.9754178957718781



In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
from datetime import datetime

then= datetime.now()
predictors = fd.getPredictors(dfTrx)

x_train, x_test, y_train, y_test, scaler =fd.split_data(dfTrx,predictors)

modelClf = ExtraTreesClassifier(random_state=42)
parameters={'n_estimators':n_estimatorsFound,
    'max_depth': max_depthFound, 
            'min_samples_leaf': min_samples_leafFound, 
            'min_samples_split': min_samples_splitFound,
           }

modelClf.set_params(**parameters)

modelClf.fit(x_train, y_train)
predsTrain = modelClf.predict(x_train)
predsTest = modelClf.predict(x_test)

now = datetime.now()
duration= now - then
duration_in_s = duration.total_seconds()
print("Duration ",duration_in_s)
resultMd.update_time_response_result('07-sklearn.ensemble.a-bagging','ExtraTreesClassifier','2-After tuning', duration_in_s)
f1,mcc,roc=fd.print_scores(y_test, predsTest,'all')
resultMd.update_performance_test_result('07-sklearn.ensemble.a-bagging','ExtraTreesClassifier','2-After tuning', f1,mcc,roc)

fd.print_scores(y_train, predsTrain,'f1', False)
fd.print_scores(y_test, predsTest,'f1')
fd.show_importance(modelClf, predictors)
fd.show_confusion_matrix(y_test, predsTest)
fd.show_prediction_graph(modelClf, x_test,y_test,'../imgs/FD07A-BaggingClassifierClassifierProbaHistogram' if saveImage else None)



In [ ]:
files = fd.getAllFiles()

range = []
f1s = []
rocs = []
loop =0
for file in files:
    loop=loop+1
    range.append(loop)
    print(file)
  
    dfTrx = pd.read_csv('../data/cleaned/'+source+file)
    predictors = fd.getPredictors(dfTrx)

    preds = modelClf.predict(dfTrx[predictors])

    f1,mcc,roc= fd.print_scores(dfTrx['Class'], preds,'All', True)
    #fd.show_importance(modelClf,predictors)
    fd.show_confusion_matrix(dfTrx['Class'], preds)
    f1s.append(f1)
    rocs.append(roc)

fd.plt_train_test(range, f1s)
resultMd.update_performance_nextdays_result(packageName,classifierName,extraParameterName, f1s[0],f1s[1],f1s[2],f1s[3],rocs[0],rocs[1],rocs[2],rocs[3])

